In [1]:
!pip install torch transformers sentencepiece datasets sacrebleu accelerate pyvi laonlp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from datasets import Dataset, concatenate_datasets
from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
import random

from pyvi import ViTokenizer
from laonlp.tokenize import word_tokenize as lao_word_tokenize

print(torch.cuda.get_device_name(0))
print("VRAM:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

NVIDIA H200
VRAM: 150.0217344 GB


In [4]:
def tokenize_vietnamese(text):
    """Tokenize Vietnamese text using PyVi"""
    try:
        return ViTokenizer.tokenize(text)
    except Exception as e:
        print(f"Error tokenizing Vietnamese: {e}")
        return text

def tokenize_lao(text):
    """Tokenize Lao text using LaoNLP"""
    try:
        # LaoNLP word_tokenize không có parameter engine
        tokens = lao_word_tokenize(text)
        return " ".join(tokens)
    except Exception as e:
        print(f"Error tokenizing Lao: {e}")
        return text

def tokenize_batch_vietnamese(texts):
    """Batch tokenize Vietnamese texts"""
    print(f"Tokenizing {len(texts)} Vietnamese texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_vietnamese(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Vietnamese texts")
    return results

def tokenize_batch_lao(texts):
    """Batch tokenize Lao texts"""
    print(f"Tokenizing {len(texts)} Lao texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_lao(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Lao texts")
    return results

print("Vietnamese and Lao tokenizers loaded successfully!")

Vietnamese and Lao tokenizers loaded successfully!


In [5]:
# ============================================================
# CELL 1: Load Best Model from Phase 2
# ============================================================
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(
    "./lo_to_vi/phase2/best"
).cuda()

tokenizer = AutoTokenizer.from_pretrained(
    "./lo_to_vi/phase2/best"
)

print("Phase 2 model loaded!")

Phase 2 model loaded!


In [6]:
# ============================================================
# CELL 2: Unfreeze Top 6 Encoder Layers (layers 6-11)
# ============================================================
# 1. Freeze toàn bộ encoder trước
for param in model.model.encoder.parameters():
    param.requires_grad = False

# 2. Unfreeze top 6 layers
for i in range(6, 12):
    for param in model.model.encoder.layers[i].parameters():
        param.requires_grad = True

# Verify
total, frozen, trainable = 0, 0, 0
encoder_trainable = 0

for name, param in model.named_parameters():
    n = param.numel()
    total += n
    if not param.requires_grad:
        frozen += n
    else:
        trainable += n
        if "encoder" in name:
            encoder_trainable += n

print(f"Total params       : {total/1e6:.1f}M")
print(f"Frozen params      : {frozen/1e6:.1f}M")
print(f"Trainable params   : {trainable/1e6:.1f}M")
print(f"Encoder trainable  : {encoder_trainable/1e6:.1f}M")

Total params       : 483.9M
Frozen params      : 206.8M
Trainable params   : 277.1M
Encoder trainable  : 126.0M


In [8]:
DATA_DIR = "dataset"
def load_parallel(src_file, tgt_file):
    with open(src_file, encoding="utf-8") as f:
        src = [l.strip() for l in f]
    with open(tgt_file, encoding="utf-8") as f:
        tgt = [l.strip() for l in f]
    
    assert len(src) == len(tgt)
    
    # Apply language-specific tokenization
    print("Tokenizing Vietnamese texts...")
    src_tokenized = tokenize_batch_lao(src)
    
    print("Tokenizing Lao texts...")
    tgt_tokenized = tokenize_batch_vietnamese(tgt)
    
    return Dataset.from_dict({
        "src_text": src_tokenized,
        "tgt_text": tgt_tokenized
    })


In [9]:
# Load toàn bộ dữ liệu từ train.vi và train.lo
full_dataset = load_parallel(
    f"{DATA_DIR}/train_lo_to_vi_shuf.lo",
    f"{DATA_DIR}/train_lo_to_vi_shuf.vi"
)

print(f"Total dataset size: {len(full_dataset)} examples")

# Chia dataset: 
# - Test: 1000 dòng cuối cùng
# - Dev: 3000 dòng trước test
# - Train: phần còn lại

test_size = 1000
dev_size = 3000

# Tính chỉ số
test_start_idx = len(full_dataset) - test_size
dev_start_idx = test_start_idx - dev_size

# Chia dataset
test_dataset = full_dataset.select(range(test_start_idx, len(full_dataset)))
dev_dataset = full_dataset.select(range(dev_start_idx, test_start_idx))
train_dataset = full_dataset.select(range(0, dev_start_idx))

# Shuffle training data để tránh bias thứ tự
train_dataset = train_dataset.shuffle(seed=42)

print(f"Train dataset: {len(train_dataset)} examples (for training)")
print(f"Dev dataset  : {len(dev_dataset)} examples (for validation during training)")
print(f"Test dataset : {len(test_dataset)} examples (for final evaluation)")
print("\nData split and shuffle completed.")

Tokenizing Vietnamese texts...
Tokenizing 700000 Lao texts...
  Processed 10000/700000 Lao texts
  Processed 20000/700000 Lao texts
  Processed 30000/700000 Lao texts
  Processed 40000/700000 Lao texts
  Processed 50000/700000 Lao texts
  Processed 60000/700000 Lao texts
  Processed 70000/700000 Lao texts
  Processed 80000/700000 Lao texts
  Processed 90000/700000 Lao texts
  Processed 100000/700000 Lao texts
  Processed 110000/700000 Lao texts
  Processed 120000/700000 Lao texts
  Processed 130000/700000 Lao texts
  Processed 140000/700000 Lao texts
  Processed 150000/700000 Lao texts
  Processed 160000/700000 Lao texts
  Processed 170000/700000 Lao texts
  Processed 180000/700000 Lao texts
  Processed 190000/700000 Lao texts
  Processed 200000/700000 Lao texts
  Processed 210000/700000 Lao texts
  Processed 220000/700000 Lao texts
  Processed 230000/700000 Lao texts
  Processed 240000/700000 Lao texts
  Processed 250000/700000 Lao texts
  Processed 260000/700000 Lao texts
  Processed

In [10]:
MAX_LEN = 256

def preprocess(batch):
    tokenizer.src_lang = "lo"
    tokenizer.tgt_lang = "vi"

    inputs = tokenizer(
        batch["src_text"],
        truncation=True,
        max_length=MAX_LEN
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt_text"],
            truncation=True,
            max_length=MAX_LEN
        )

    inputs["labels"] = labels["input_ids"]
    return inputs


In [11]:
train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=8
)

dev_dataset = dev_dataset.map(
    preprocess,
    batched=True,
    remove_columns=dev_dataset.column_names,
    num_proc=8
)


Map (num_proc=8):   0%|          | 0/696000 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

Map (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

In [12]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True
)


In [13]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [14]:
# ============================================================
# CELL 3: Training Arguments - Phase 3
# ============================================================
training_args_phase3 = TrainingArguments(
    output_dir="./lo_to_vi/phase3",
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=500,
    
    # Saving
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    
    # Logging
    logging_steps=100,
    
    # Batch size
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    
    # Learning rate - LOWER than phase 2
    learning_rate=3e-5,  # Lower for fine-tuning encoder
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.05,
    
    # Regularization
    weight_decay=0.015,
    max_grad_norm=0.8,
    
    # Precision
    num_train_epochs=6,
    
    # FP16
    bf16=True,
    fp16=False,

    # Speed
    group_by_length=True,
    dataloader_num_workers=8,
    dataloader_pin_memory=True,
    
    # Best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    report_to="none",
)


In [15]:
# ============================================================
# CELL 4: Create Trainer - Phase 3
# ============================================================
trainer_phase3 = Trainer(
    model=model,
    args=training_args_phase3,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=4)
    ]
)


/tmp/ipykernel_4534/2611886879.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_phase3 = Trainer(


In [16]:
# ============================================================
# CELL 5: Train Phase 3
# ============================================================
print("\n" + "="*60)
print("PHASE 3: Progressive Unfreezing (Top 6 Encoder Layers)")
print("="*60 + "\n")

trainer_phase3.train()


PHASE 3: Progressive Unfreezing (Top 6 Encoder Layers)



Step,Training Loss,Validation Loss
500,0.773300,0.812738
1000,0.809600,0.813648
1500,0.809100,0.806197
2000,0.810200,0.804188
2500,0.814400,0.798868
3000,0.774800,0.791497
3500,0.779700,0.787545
4000,0.771000,0.783219
4500,0.774800,0.778470
5000,0.772000,0.772852


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=16314, training_loss=0.732884539624881, metrics={'train_runtime': 3300.9116, 'train_samples_per_second': 1265.105, 'train_steps_per_second': 4.942, 'total_flos': 5.62798690496938e+17, 'train_loss': 0.732884539624881, 'epoch': 6.0})

In [17]:
# ============================================================
# CELL 6: Save Phase 3 Model
# ============================================================
trainer_phase3.save_model("./lo_to_vi/phase3/best")
tokenizer.save_pretrained("./lo_to_vi/phase3/best")

print("\nPhase 3 model saved!")



Phase 3 model saved!


In [20]:
# ============================================================
# CELL 6: Evaluate Phase 3
# ============================================================
def translate_batch(texts, model, tokenizer, batch_size=32):
    """Batch translation for speed"""
    model.eval()
    tokenizer.src_lang = "lo"
    tokenizer.tgt_lang = "vi"
    
    outputs = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to("cuda")
        
        with torch.no_grad():
            gen = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.get_lang_id("vi"),
                num_beams=5,
                max_length=256
            )
        
        texts_out = tokenizer.batch_decode(gen, skip_special_tokens=True)
        outputs.extend(texts_out)
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"Translated {i+len(batch)}/{len(texts)}")
    
    return outputs

# Lấy dữ liệu test từ test_dataset (đã chia từ train.vi/train.lo)
test_lo = test_dataset["src_text"]
test_vi = test_dataset["tgt_text"]

print(f"\nTest set size: {len(test_vi)} examples")
print("\nTranslating test set...")
preds_phase3 = translate_batch(test_lo, model, tokenizer)




Test set size: 1000 examples

Translating test set...
Translated 320/1000
Translated 640/1000
Translated 960/1000


In [22]:
from sacrebleu import corpus_bleu

# Calculate BLEU
bleu_phase3 = corpus_bleu(preds_phase3, [test_vi])
print(f"\n{'='*60}")
print(f"PHASE 3 BLEU Score: {bleu_phase3.score:.2f}")
print(f"{'='*60}\n")

# Save predictions
with open("./lo_to_vi/phase3/phase3_predictions.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(preds_phase3))

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.



PHASE 3 BLEU Score: 50.76



In [25]:
# ============================================================
# CELL 15: Sample Translations
# ============================================================
print("\n" + "="*70)
print("SAMPLE TRANSLATIONS")
print("="*70)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Source    : {test_lo[i]}")
    print(f"Reference : {test_vi[i]}")
    print(f"Prediction: {preds_phase3[i]}")
    print("-" * 70)



SAMPLE TRANSLATIONS

Example 1:
Source    : ພຽງ ແຕ່ ການຈັບ ຄູ່ ກະໂປງ ສັ້ນ ລາຍ ດອກ ກັບ ເສື້ອ ເປີດ ບ່າໄຫລ່ ກໍ ຈະ ເຮັດໃຫ້ ສາວ ໆ ມີ ຊຸດ ທີ່ເບິ່ງ ດີ ແລະ ເທ່ ທັນທີ .
Reference : Chỉ việc mix chân váy hoa dáng ngắn cùng áo trễ vai , các cô gái có ngay một set đồ " hack " dáng lại mát_rượi .
Prediction: Chỉ cần kết_hợp chân váy ngắn hoa với áo hở vai là các cô nàng sẽ có ngay set đồ đẹp_mắt , nuột_nà .
----------------------------------------------------------------------

Example 2:
Source    : ຮູບເງົາ ບາງ ເລື່ອງ ທີ່   Phuong   Thanh   ໄດ້ ເຂົ້າຮ່ວມ ສະແດງ ລວມ ມີ :   Walking   and   Crying ,   Surrogate   Mother ,   When   Men   Get   Pregnant ,   The   Soul   of   Truong   Ba ,   the   Butcher ' s   Skin ,   The   Kiss   of   Death ,   Beautiful   Every   Centimeter ,   Rescuing   the   God   of   Death ,   ແລະ   Hot   Boy   Rebellion .
Reference : Các bộ phim Phương_Thanh đã tham_gia như : Vừa đi vừa khóc , Đẻ_mướn , Khi đàn_ông có bầu , Hồn Trương Ba , da hàng thịt , Nụ hôn thần_chết , Đẹp